In [1]:
import os
import shutil
import os.path as osp

import torch
from torch import nn
import torch.nn.functional as F

from accelerate import Accelerator
from accelerate.utils import LoggerType

from transformers import AdamW, AlbertConfig, AlbertModel, AutoTokenizer
from accelerate import DistributedDataParallelKwargs

from model import MultiTaskModel
from dataloader import build_dataloader
from utils import length_to_mask, scan_checkpoint

from datasets import load_dataset

from torch.utils.tensorboard import SummaryWriter

/root/miniconda3/envs/pl-bert/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/pl-bert/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
import yaml
import pickle

config_path = "Configs/config.yml"
config = yaml.safe_load(open(config_path))

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(config['dataset_params']['tokenizer'])

In [4]:
criterion = nn.CrossEntropyLoss() # F0 loss (regression)

best_loss = float('inf')  # best test loss
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
loss_train_record = list([])
loss_test_record = list([])

num_steps = config['num_steps']
log_interval = config['log_interval']
save_interval = config['save_interval']

In [5]:
def train():
    ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)

    curr_steps = 0

    dataset = load_dataset(config["data_folder"], split="train")

    log_dir = config["log_dir"]
    if not osp.exists(log_dir):
        os.makedirs(log_dir, exist_ok=True)
    shutil.copy(config_path, osp.join(log_dir, osp.basename(config_path)))

    batch_size = config["batch_size"]
    train_loader = build_dataloader(
        dataset, batch_size=batch_size, num_workers=0, dataset_config=config["dataset_params"]
    )

    albert_base_configuration = AlbertConfig(**config["model_params"])

    bert = AlbertModel(albert_base_configuration)
    bert = MultiTaskModel(
        bert,
        num_vocab=len(tokenizer),
        num_tokens=config["model_params"]["vocab_size"],
        hidden_size=config["model_params"]["hidden_size"],
    )

    load = True
    try:
        files = os.listdir(log_dir)
        ckpts = []
        for f in os.listdir(log_dir):
            if f.startswith("step_"):
                ckpts.append(f)

        iters = [int(f.split("_")[-1].split(".")[0]) for f in ckpts if os.path.isfile(os.path.join(log_dir, f))]
        iters = sorted(iters)[-1]
    except:
        iters = 0
        load = False

    optimizer = AdamW(bert.parameters(), lr=1e-4)

    accelerator = Accelerator(
        mixed_precision=config["mixed_precision"], split_batches=True, kwargs_handlers=[ddp_kwargs]
    )

    if load:
        checkpoint = torch.load(log_dir + "/step_" + str(iters) + ".t7", map_location="cpu")
        state_dict = checkpoint["net"]
        from collections import OrderedDict

        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:]  # remove `module.`
            new_state_dict[name] = v

        bert.load_state_dict(new_state_dict, strict=False)

        accelerator.print("Checkpoint loaded.")
        optimizer.load_state_dict(checkpoint["optimizer"])

    bert, optimizer, train_loader = accelerator.prepare(bert, optimizer, train_loader)

    accelerator.print("Start training...")

    running_loss = 0

    for _, batch in enumerate(train_loader):
        curr_steps += 1

        words, labels, phonemes, input_lengths, masked_indices = batch
        text_mask = length_to_mask(torch.Tensor(input_lengths)).to(accelerator.device)

        tokens_pred, words_pred = bert(phonemes, attention_mask=(~text_mask).int())

        loss_vocab = 0
        for _s2s_pred, _text_input, _text_length, _masked_indices in zip(
            words_pred, words, input_lengths, masked_indices
        ):
            loss_vocab += criterion(_s2s_pred[:_text_length], _text_input[:_text_length])
        loss_vocab /= words.size(0)

        loss_token = 0
        sizes = 1
        for _s2s_pred, _text_input, _text_length, _masked_indices in zip(
            tokens_pred, labels, input_lengths, masked_indices
        ):
            if len(_masked_indices) > 0:
                _text_input = _text_input[:_text_length][_masked_indices]
                loss_tmp = criterion(_s2s_pred[:_text_length][_masked_indices], _text_input[:_text_length])
                loss_token += loss_tmp
                sizes += 1
        loss_token /= sizes

        loss = loss_vocab + loss_token

        optimizer.zero_grad()
        accelerator.backward(loss)
        optimizer.step()

        running_loss += loss.item()

        iters = iters + 1
        if (iters + 1) % log_interval == 0:
            accelerator.print(
                "Step [%d/%d], Loss: %.5f, Vocab Loss: %.5f, Token Loss: %.5f"
                % (iters + 1, num_steps, running_loss / log_interval, loss_vocab, loss_token)
            )
            running_loss = 0

        if (iters + 1) % save_interval == 0:
            accelerator.print("Saving..")

            state = {
                "net": bert.state_dict(),
                "step": iters,
                "optimizer": optimizer.state_dict(),
            }

            accelerator.save(state, log_dir + "/step_" + str(iters + 1) + ".t7")

        if curr_steps > num_steps:
            return

In [6]:
from accelerate import notebook_launcher
while True:
    notebook_launcher(train, args=(), num_processes=1, use_port=33389)

Launching training on one GPU.


Generating train split: 100%|██████████| 9839572/9839572 [01:51<00:00, 88531.57 examples/s] 
/root/miniconda3/envs/pl-bert/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/root/miniconda3/envs/pl-bert/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=True)
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Start training...
Step [10/1000000], Loss: 10.99973, Vocab Loss: 8.78823, Token Loss: 2.81447
Step [20/1000000], Loss: 11.06915, Vocab Loss: 8.04384, Token Loss: 2.95215
Step [30/1000000], Loss: 10.77548, Vocab Loss: 8.36543, Token Loss: 2.69089
Step [40/1000000], Loss: 10.41517, Vocab Loss: 7.35178, Token Loss: 2.79306
Step [50/1000000], Loss: 10.11564, Vocab Loss: 7.03958, Token Loss: 2.87688
Step [60/1000000], Loss: 10.05284, Vocab Loss: 6.80095, Token Loss: 2.66740
Step [70/1000000], Loss: 9.49882, Vocab Loss: 6.52335, Token Loss: 3.01062
Step [80/1000000], Loss: 9.43032, Vocab Loss: 5.90150, Token Loss: 2.67500
Step [90/1000000], Loss: 9.88101, Vocab Loss: 7.3391